In [1]:
import pandas as pd 
import numpy as np 


In [2]:
data_df = pd.read_parquet(r'D:\python\financial statement prj\chatbot_financial_statement\csv\v3\corp_financial_report.parquet')
# data_df = pd.read_csv(r'D:\python\financial statement prj\group_2_chatbot_financial_statement\sample\sample_data\non_bank_financial_report.csv')


In [3]:
# map_df.head()

In [4]:
data_df.head()

,category_code,data,stock_code,quarter,year,date_added
0,IS_001,1.696912e+07,HSG,0,2015,2015-12-30
1,IS_001,1.985372e+07,HSG,0,2016,2016-12-30
2,IS_001,2.847371e+07,HSG,0,2017,2017-12-30
3,IS_001,3.419811e+07,HSG,0,2018,2018-12-30
4,IS_001,2.712138e+07,HSG,0,2019,2019-12-30


## Phân tích cấu trúc tài chính 

In [5]:
import pandas as pd

# Pivot table to restructure financial data based on category codes
pivot_df = data_df.pivot_table(index=['stock_code', 'year', 'quarter'], 
                                           columns='category_code', 
                                           values='data', 
                                           aggfunc='sum')

# Ratio calculation functions
def EBIT(income_before_tax, interest_expense = None):
    if interest_expense is None:
        return income_before_tax
    return income_before_tax + interest_expense

def equity_ratio(equity, total_assets):
    return equity / total_assets if total_assets else None

def long_term_asset_self_financing_ratio(permanent_capital, long_term_assets):
    return permanent_capital / long_term_assets if long_term_assets else None

def fixed_asset_self_financing_ratio(permanent_capital, fixed_assets):
    return permanent_capital / fixed_assets if fixed_assets else None

def general_solvency_ratio(total_assets, total_liabilities):
    return total_assets / total_liabilities if total_liabilities else None

def return_on_investment(net_income, total_investment):
    return net_income / total_investment if total_investment else None

def ROIC(NOPAT, invested_capital):
    return NOPAT / invested_capital if invested_capital else None

def return_on_long_term_capital(EBIT, average_long_term_capital):
    return EBIT / average_long_term_capital if average_long_term_capital else None

def basic_earning_power(EBIT, average_total_assets):
    return EBIT / average_total_assets if average_total_assets else None

def debt_to_assets_ratio(total_liabilities, total_assets):
    return total_liabilities / total_assets if total_assets else None

def debt_to_equity_ratio(total_liabilities, equity):
    return total_liabilities / equity if equity else None

def short_term_debt_to_assets_ratio(short_term_liabilities, total_assets):
    return short_term_liabilities / total_assets if total_assets else None

def interest_coverage_ratio(EBIT, interest_expense):
    return EBIT / interest_expense if interest_expense else None

def long_term_debt_to_equity_ratio(long_term_liabilities, equity):
    return long_term_liabilities / equity if equity else None

def short_term_debt_to_equity_ratio(short_term_liabilities, equity):
    return short_term_liabilities / equity if equity else None

# Dictionary to hold functions and corresponding category codes
ratio_functions = {
    'EBIT': ['IS_050', 'IS_023'],  # income_before_tax, interest_expense
    'equity_ratio': ['BS_400', 'BS_270'],  # equity, total_assets
    'long_term_asset_self_financing_ratio': [['BS_400', 'BS_330'], 'BS_200'],  # permanent_capital (equity + long_term_liabilities), long_term_assets
    'fixed_asset_self_financing_ratio': [['BS_400', 'BS_330'], ['BS_220','BS_240']],  # permanent_capital (equity + long_term_liabilities), fixed_assets
    'general_solvency_ratio': ['BS_270', 'BS_300'],  #  total_assets, total_liabilities
    'return_on_investment': ['IS_060', 'BS_270'],  # net_income, total_investment (using total assets as example)
    # 'ROIC': ['IS_060', 'BS_200'],  # NOPAT, invested_capital
    # 'return_on_long_term_capital': [['IS_050','IS_023], 'BS_330'],  # EBIT, long_term_liabilities
    'basic_earning_power': [['IS_050','IS_023'], 'BS_270'],  # EBIT, total_assets
    'debt_to_assets_ratio': ['BS_300', 'BS_270'],  # total_liabilities, total_assets
    'debt_to_equity_ratio': ['BS_300', 'BS_400'],  # total_liabilities, equity
    'short_term_debt_to_assets_ratio': ['BS_310', 'BS_270'],  # short_term_liabilities, total_assets
    'interest_coverage_ratio': [['IS_050','IS_023'], 'IS_023'],  # EBIT, interest_expense
    'long_term_debt_to_equity_ratio': ['BS_330', 'BS_400'],  # long_term_liabilities, equity
    'short_term_debt_to_equity_ratio': ['BS_310', 'BS_400']  # short_term_liabilities, equity
}

# Create a DataFrame to store the results
results_1 = []

# Iterate through pivoted data
for index, row in pivot_df.iterrows():
    stock_code, year, quarter = index
    
    for ratio, inputs in ratio_functions.items():
        # Fetch input data from the pivot table
        input_values = []
        for input_name in inputs:
            if isinstance(input_name, list):  # For cases like permanent capital (sum of multiple values)
                # Sum multiple values if input_name is a list
                value_sum = sum([row[i] for i in input_name if i in row.index])
                input_values.append(value_sum)
            else:
                input_values.append(row[input_name] if input_name in row.index else None)
        
        # Check if all required data is available
        if None not in input_values:
            # Call the corresponding function to calculate the ratio
            ratio_value = globals()[ratio](*input_values)
            results_1.append({
                'stock_code': stock_code,
                'year': year,
                'quarter': quarter,
                'ratio_code': ratio,
                'data': ratio_value
            })

# Convert results to DataFrame
ratios_df_1 = pd.DataFrame(results_1)

ratios_df_1


,stock_code,year,quarter,ratio_code,data
0,ACV,2015,0,EBIT,2.187559e+06
1,ACV,2015,0,equity_ratio,4.596180e-01
2,ACV,2015,0,long_term_asset_self_financing_ratio,1.593160e+00
3,ACV,2015,0,fixed_asset_self_financing_ratio,1.690095e+00
4,ACV,2015,0,general_solvency_ratio,1.850543e+00
...,...,...,...,...,...
47679,YEG,2024,3,debt_to_equity_ratio,6.889008e-01
47680,YEG,2024,3,short_term_debt_to_assets_ratio,3.826225e-01
47681,YEG,2024,3,interest_coverage_ratio,-3.227513e+00
47682,YEG,2024,3,long_term_debt_to_equity_ratio,4.268928e-02


In [6]:
VNM_equity_ratio_q1_2024 = ratios_df_1[
    (ratios_df_1['stock_code'] == 'VNM') & 
    (ratios_df_1['year'] == 2024) & 
    (ratios_df_1['quarter'] == 1) & 
    (ratios_df_1['ratio_code'] == 'equity_ratio')
]['data'].values[0]

print(f"Equity Ratio of VNM in Q1 2024: {VNM_equity_ratio_q1_2024}")

Equity Ratio of VNM in Q1 2024: 0.6943834178155656


## Phân tích khả năng thanh toán (LIquidity)

In [7]:
import pandas as pd

# Pivot table to restructure financial data based on category codes
pivot_df_2 = data_df.pivot_table(index=['stock_code', 'year', 'quarter'], 
                                           columns='category_code', 
                                           values='data', 
                                           aggfunc='sum')

# Ratio calculation functions
def receivables_to_payables_ratio(accounts_receivable, total_liabilities):
    return accounts_receivable / total_liabilities if total_liabilities else None

def receivables_to_total_assets_ratio(accounts_receivable, total_assets):
    return accounts_receivable / total_assets if total_assets else None

def debt_to_total_capital_ratio(total_liabilities, total_capital):
    return total_liabilities / total_capital if total_capital else None

def receivables_to_sales_ratio(accounts_receivables, total_sales):
    return accounts_receivables / total_sales if total_sales else None

def allowance_for_doubtful_accounts_ratio(allowance_for_doubtful_accounts, accounts_receivables):
    return allowance_for_doubtful_accounts / accounts_receivables if accounts_receivables else None

def asset_to_debt_ratio(total_assets, total_liabilities):
    return total_assets / total_liabilities if total_liabilities else None

def current_ratio(current_assets, current_liabilities):
    return current_assets / current_liabilities if current_liabilities else None

def quick_ratio(current_assets, inventory, current_liabilities):
    return (current_assets - inventory) / current_liabilities if current_liabilities else None

def cash_ratio(cash_and_cash_equivalents, current_liabilities):
    return cash_and_cash_equivalents / current_liabilities if current_liabilities else None

def long_term_debt_coverage_ratio(non_current_assets, non_current_liabilities):
    return non_current_assets / non_current_liabilities if non_current_liabilities else None

def debt_to_equity_ratio(total_liabilities, total_equity):
    return total_liabilities / total_equity if total_equity else None

def long_term_debt_to_equity_capital_ratio(non_current_liabilities, equity):
    return non_current_liabilities / equity if equity else None

def time_interest_earned(EBIT, interest_expense):
    return EBIT / interest_expense if interest_expense else None

def debt_to_tangible_net_worth_ratio(total_liabilities, equity, intangible_assets):
    return total_liabilities / (equity - intangible_assets) if (equity - intangible_assets) else None

ratio_functions = {
    'receivables_to_payables_ratio': [['BS_130', 'BS_210'], 'BS_300'],  # accounts_receivable, total_liabilities
    'receivables_to_total_assets_ratio': [['BS_130', 'BS_210'], 'BS_270'],  # accounts_receivable, total_assets
    'debt_to_total_capital_ratio': ['BS_300', 'BS_440'],  # total_liabilities, total_capital
    'receivables_to_sales_ratio': [['BS_131', 'BS_211'], 'IS_010'],  # accounts_receivables, total_sales
    'allowance_for_doubtful_accounts_ratio': [['BS_137', 'BS_219'], ['BS_131', 'BS_211']],  # allowance_for_doubtful_accounts, accounts_receivables
    'asset_to_debt_ratio': ['BS_270', 'BS_300'],  # total_assets, total_liabilities
    'current_ratio': [['BS_100', 'BS_151'], 'BS_310'],  # current_assets_for_liquidity, current_liabilities
    'quick_ratio': [['BS_100', 'BS_151'], 'BS_140', 'BS_310'],  # current_assets - inventory, current_liabilities
    'cash_ratio': ['BS_110', 'BS_310'],  # cash_and_cash_equivalents, current_liabilities
    'long_term_debt_coverage_ratio': ['BS_200', 'BS_330'],  # non_current_assets, non_current_liabilities
    'debt_to_equity_ratio': ['BS_300', 'BS_400'],  # total_liabilities, total_equity
    'long_term_debt_to_equity_capital_ratio': ['BS_330', 'BS_400'],  # long_term_liabilities, equity
    'time_interest_earned': [['IS_050', 'IS_023'], 'IS_023'],  # EBIT, interest_expense
    'debt_to_tangible_net_worth_ratio': ['BS_300', 'BS_400', 'BS_227'],  # total_liabilities, equity, intangible_assets
}

# Create a DataFrame to store the results
results_2 = []

# Iterate through pivoted data
for index, row in pivot_df_2.iterrows():
    stock_code, year, quarter = index
    
    for ratio, inputs in ratio_functions.items():
        input_values = []
        if ratio == 'current_ratio':
            # Subtract BS_151 from BS_100 for current ratio calculation
            current_assets = row[inputs[0][0]] - row[inputs[0][1]] if inputs[0][0] in row.index and inputs[0][1] in row.index else None
            input_values.append(current_assets)
            input_values.append(row[inputs[1]] if inputs[1] in row.index else None)
        elif ratio == 'quick_ratio':
            # Subtract BS_140 from BS_100 for quick ratio calculation
            current_assets = row[inputs[0][0]] - row[inputs[0][1]] if inputs[0][0] in row.index and inputs[0][1] in row.index else None
            inventory = row[inputs[1]] if inputs[1] in row.index else None
            current_liabilities = row[inputs[2]] if inputs[2] in row.index else None
            input_values.extend([current_assets, inventory, current_liabilities])
        else:
            # For all other ratios, sum values if needed or fetch directly
            for input_name in inputs:
                if isinstance(input_name, list):
                    value_sum = sum([row[i] for i in input_name if i in row.index])
                    input_values.append(value_sum)
                else:
                    input_values.append(row[input_name] if input_name in row.index else None)
        
        # Check if all required data is available
        if None not in input_values:
            # Call the corresponding function to calculate the ratio
            ratio_value = globals()[ratio](*input_values)
            results_2.append({
                'stock_code': stock_code,
                'year': year,
                'quarter': quarter,
                'ratio_code': ratio,
                'data': ratio_value
            })

# Convert results to DataFrame
ratios_df_2 = pd.DataFrame(results_2)

# Display or export the DataFrame
ratios_df_2


,stock_code,year,quarter,ratio_code,data
0,ACV,2015,0,receivables_to_payables_ratio,0.150912
1,ACV,2015,0,receivables_to_total_assets_ratio,0.081550
2,ACV,2015,0,debt_to_total_capital_ratio,0.540382
3,ACV,2015,0,receivables_to_sales_ratio,0.117975
4,ACV,2015,0,allowance_for_doubtful_accounts_ratio,-0.214929
...,...,...,...,...,...
51347,YEG,2024,3,long_term_debt_coverage_ratio,19.466969
51348,YEG,2024,3,debt_to_equity_ratio,0.688901
51349,YEG,2024,3,long_term_debt_to_equity_capital_ratio,0.042689
51350,YEG,2024,3,time_interest_earned,-3.227513


## Phân tích rủi ro tài chính (Financial risk)

In [8]:
import pandas as pd

# Create a new pivot table for the new ratios
pivot_df_3 = data_df.pivot_table(index=['stock_code', 'year', 'quarter'], 
                                 columns='category_code', 
                                 values='data', 
                                 aggfunc='sum')

# Financial Leverage = total_liabilities(BS_300) / total_lia_and_equity (BS_440)
def financial_leverage(total_liabilities, total_lia_and_equity):
    return total_liabilities / total_lia_and_equity if total_lia_and_equity else None

# Allowance for Doubtful Accounts to Total Assets Ratio = allowance_for_doubtful_accounts(BS_137+BS_219) / total_assets (BS_270)
def allowance_for_doubtful_accounts_to_total_assets_ratio(allowance_for_doubtful_accounts, total_assets):
    return allowance_for_doubtful_accounts / total_assets if total_assets else None

# Permanent Financing Ratio (Hệ số tài trợ thường xuyên) = permanent_capital(BS_400 + BS_330) / total_lia_and_equity (BS_440)
def permanent_financing_ratio(permanent_capital, total_lia_and_equity):
    return permanent_capital / total_lia_and_equity if total_lia_and_equity else None

new_ratio_functions = {
    'financial_leverage': ['BS_300', 'BS_440'],  # total_liabilities (BS_300), total_lia_and_equity (BS_440)
    'allowance_for_doubtful_accounts_to_total_assets_ratio': [['BS_137', 'BS_219'], 'BS_270'],  # allowance_for_doubtful_accounts (BS_137+BS_219), total_assets (BS_270)
    'permanent_financing_ratio': [['BS_400', 'BS_330'], 'BS_440'],  # permanent_capital (BS_400 + BS_330), total_lia_and_equity (BS_440)
}

# Create a DataFrame to store the results of the new ratios
results_3 = []

# Iterate through the pivot table to calculate the new ratios
for index, row in pivot_df_3.iterrows():
    stock_code, year, quarter = index
    
    for ratio, inputs in new_ratio_functions.items():
        input_values = []
        for input_name in inputs:
            if isinstance(input_name, list):  # Sum for cases like permanent capital or allowance for doubtful accounts
                value_sum = sum([row[i] for i in input_name if i in row.index])
                input_values.append(value_sum)
            else:
                input_values.append(row[input_name] if input_name in row.index else None)
        
        # Check if all required data is available
        if None not in input_values:
            # Call the corresponding function to calculate the ratio
            ratio_value = globals()[ratio](*input_values)
            results_3.append({
                'stock_code': stock_code,
                'year': year,
                'quarter': quarter,
                'ratio_code': ratio,
                'data': ratio_value
            })

# Convert the results to a DataFrame
ratios_df_3 = pd.DataFrame(results_3)

# Display or export the DataFrame
ratios_df_3


,stock_code,year,quarter,ratio_code,data
0,ACV,2015,0,financial_leverage,0.540382
1,ACV,2015,0,allowance_for_doubtful_accounts_to_total_asset...,-0.007447
2,ACV,2015,0,permanent_financing_ratio,0.818265
3,ACV,2016,0,financial_leverage,0.481645
4,ACV,2016,0,allowance_for_doubtful_accounts_to_total_asset...,-0.006476
...,...,...,...,...,...
10999,YEG,2024,2,allowance_for_doubtful_accounts_to_total_asset...,-0.042439
11000,YEG,2024,2,permanent_financing_ratio,0.735017
11001,YEG,2024,3,financial_leverage,0.407899
11002,YEG,2024,3,allowance_for_doubtful_accounts_to_total_asset...,-0.036184


## phân tích kết quả kinh doanh 

In [9]:
# # Doanh thu thuần về bán hàng và cung cấp dịch vụ ,IS_010
# # Doanh thu hoạt động tài chính,IS_021,Financial income
# # doanh thu thuần = IS_010 + IS_021

# # Financial Income to Net Revenue Ratio = Financial Income(IS_021)/ Net Revenue
# def financial_income_to_net_revenue_ratio(financial_income, net_revenue):
#     return financial_income / net_revenue



In [10]:
import pandas as pd

# Create a new pivot table for the Financial Income to Net Revenue Ratio
pivot_df_4 = data_df.pivot_table(index=['stock_code', 'year', 'quarter'], 
                                 columns='category_code', 
                                 values='data', 
                                 aggfunc='sum')

# Financial Income to Net Revenue Ratio = Financial Income(IS_021) / Net Revenue
def financial_income_to_net_revenue_ratio(financial_income, net_revenue):
    return financial_income / net_revenue if net_revenue else None

# Dictionary to hold the category codes for the new ratio
new_ratio_function = {
    'financial_income_to_net_revenue_ratio': ['IS_021', 'IS_010']  # financial_income (IS_021), net_revenue (IS_010)
}

# Create a DataFrame to store the results of the new ratio
results_4 = []

# Iterate through the pivot table to calculate the new ratio
for index, row in pivot_df_4.iterrows():
    stock_code, year, quarter = index
    
    for ratio, inputs in new_ratio_function.items():
        input_values = []
        for input_name in inputs:
            input_values.append(row[input_name] if input_name in row.index else None)
        
        # Check if all required data is available
        if None not in input_values:
            # Call the corresponding function to calculate the ratio
            ratio_value = globals()[ratio](*input_values)
            results_4.append({
                'stock_code': stock_code,
                'year': year,
                'quarter': quarter,
                'ratio_code': ratio,
                'data': ratio_value
            })

# Convert the results to a DataFrame
ratios_df_4 = pd.DataFrame(results_4)

# Display or export the DataFrame
ratios_df_4


,stock_code,year,quarter,ratio_code,data
0,ACV,2015,0,financial_income_to_net_revenue_ratio,0.072857
1,ACV,2016,0,financial_income_to_net_revenue_ratio,0.094544
2,ACV,2016,1,financial_income_to_net_revenue_ratio,0.063617
3,ACV,2016,2,financial_income_to_net_revenue_ratio,0.056813
4,ACV,2016,3,financial_income_to_net_revenue_ratio,0.055858
...,...,...,...,...,...
3663,YEG,2023,3,financial_income_to_net_revenue_ratio,0.096529
3664,YEG,2023,4,financial_income_to_net_revenue_ratio,0.075837
3665,YEG,2024,1,financial_income_to_net_revenue_ratio,0.491453
3666,YEG,2024,2,financial_income_to_net_revenue_ratio,0.194149


## phân tích khả năng sinh lời ( profitability )

In [11]:
import pandas as pd

# Create a new pivot table for the ratios in pivot_df_5
pivot_df_5 = data_df.pivot_table(index=['stock_code', 'year', 'quarter'], 
                                 columns='category_code', 
                                 values='data', 
                                 aggfunc='sum')

# Ratio calculation functions

def return_on_assets(net_income, total_assets):
    return net_income / total_assets if total_assets else None

def return_on_fixed_assets(net_income, average_fixed_assets):
    return net_income / average_fixed_assets if average_fixed_assets else None

def return_on_long_term_operating_assets(net_income, average_long_term_operating_assets):
    return net_income / average_long_term_operating_assets if average_long_term_operating_assets else None

def Basic_Earning_Power_Ratio(EBIT, total_assets):
    return EBIT / total_assets if total_assets else None

def Return_on_equity(net_income, equity):
    return net_income / equity if equity else None

# def return_on_common_equity(net_income, preferred_dividends, average_common_equity):
#     return (net_income - preferred_dividends) / average_common_equity if average_common_equity else None

def profitability_of_cost_of_goods_sold(net_income_from_operating, COGS):
    return net_income_from_operating / COGS if COGS else None

def price_spread_ratio(gross_profit, COGS):
    return gross_profit / COGS if COGS else None

def profitability_of_operating_expenses(net_income_from_operating, total_operating_expenses):
    return net_income_from_operating / total_operating_expenses if total_operating_expenses else None

def Return_on_sales(net_income, net_sales):
    return net_income / net_sales if net_sales else None

def operating_profit_margin(net_profit_from_operating, net_sales):
    return net_profit_from_operating / net_sales if net_sales else None

def gross_profit_margin(gross_profit, net_sales):
    return gross_profit / net_sales if net_sales else None

def net_profit_margin(net_income, net_sales):
    return Return_on_sales(net_income, net_sales)

# Dictionary to hold functions and corresponding category codes
ratio_functions_5 = {
    'return_on_assets': ['IS_060', 'BS_270'],  # net_income (IS_060), total_assets (BS_270)
    'return_on_fixed_assets': ['IS_060', 'BS_220'],  # net_income (IS_060), average_fixed_assets (BS_220)
    'return_on_long_term_operating_assets': ['IS_060', 'BS_240'],  # net_income (IS_060), average_long_term_operating_assets (BS_240)
    'Basic_Earning_Power_Ratio': [['IS_050', 'IS_023'], 'BS_270'],  # EBIT (IS_050 + IS_023), total_assets (BS_270)
    'Return_on_equity': ['IS_060', 'BS_400'],  # net_income (IS_060), equity (BS_400)
    # 'return_on_common_equity': ['IS_060', 'CF_036', 'BS_400'],  # net_income (IS_060), preferred_dividends (CF_036), average_common_equity (BS_400)
    'profitability_of_cost_of_goods_sold': ['IS_030', 'IS_011'],  # net_income_from_operating (IS_030), COGS (IS_011)
    'price_spread_ratio': ['IS_020', 'IS_011'],  # gross_profit (IS_020), COGS (IS_011)
    'profitability_of_operating_expenses': ['IS_030', ['IS_025', 'IS_026', 'IS_011']],  # net_income_from_operating (IS_030), total_operating_expenses (IS_025 + IS_026 + IS_011)
    'Return_on_sales': ['IS_060', ['IS_010','IS_021']],  # net_income (IS_060), net_sales (IS_010)
    'operating_profit_margin': ['IS_030',  ['IS_010','IS_021']],  # NOPAT, net_sales (IS_010)
    'gross_profit_margin': ['IS_020',  ['IS_010','IS_021']],  # gross_profit (IS_020), net_sales (IS_010)
    'net_profit_margin': ['IS_060',  ['IS_010','IS_021']]  # net_income (IS_060), net_sales (IS_010)
}

# Create a DataFrame to store the results of the new ratios
results_5 = []

# Iterate through the pivot table to calculate the new ratios
for index, row in pivot_df_5.iterrows():
    stock_code, year, quarter = index
    
    for ratio, inputs in ratio_functions_5.items():
        input_values = []
        for input_name in inputs:
            if isinstance(input_name, list):  # Sum for cases like EBIT or total operating expenses
                value_sum = sum([row[i] for i in input_name if i in row.index])
                input_values.append(value_sum)
            else:
                input_values.append(row[input_name] if input_name in row.index else None)
        
        # Check if all required data is available
        if None not in input_values:
            # Call the corresponding function to calculate the ratio
            ratio_value = globals()[ratio](*input_values)
            results_5.append({
                'stock_code': stock_code,
                'year': year,
                'quarter': quarter,
                'ratio_code': ratio,
                'data': ratio_value
            })

# Convert the results to a DataFrame
ratios_df_5 = pd.DataFrame(results_5)

# Display or export the DataFrame
ratios_df_5


,stock_code,year,quarter,ratio_code,data
0,ACV,2015,0,return_on_assets,0.039087
1,ACV,2015,0,return_on_fixed_assets,0.084850
2,ACV,2015,0,return_on_long_term_operating_assets,1.663509
3,ACV,2015,0,Basic_Earning_Power_Ratio,0.048773
4,ACV,2015,0,Return_on_equity,0.085041
...,...,...,...,...,...
44011,YEG,2024,3,profitability_of_operating_expenses,-0.115533
44012,YEG,2024,3,Return_on_sales,0.092808
44013,YEG,2024,3,operating_profit_margin,0.097651
44014,YEG,2024,3,gross_profit_margin,0.214196


## phân tích cashflow


In [12]:
import pandas as pd

# Create a new pivot table for the cash flow ratios in pivot_df_7
pivot_df_6 = data_df.pivot_table(index=['stock_code', 'year', 'quarter'], 
                                 columns='category_code', 
                                 values='data', 
                                 aggfunc='sum')

# Function to get the value of quarter 0 from the previous year for a given stock_code
def get_previous_year_q0_value(stock_code, year, category_code):
    try:
        return pivot_df_6.loc[(stock_code, year - 1, 0), category_code]
    except KeyError:
        return None

# Ratio calculation functions

def EBITDA(EBIT, depreciation_and_amortization = None):
    if depreciation_and_amortization is None:
        return EBIT
    return EBIT + depreciation_and_amortization

def free_cash_flow(operating_net_cash_flow, capital_expenditures, dividends_paid):
    return operating_net_cash_flow - capital_expenditures - dividends_paid

def free_cash_flow_to_operating_cash_flow_ratio(free_cash_flow, operating_net_cash_flow):
    return free_cash_flow / operating_net_cash_flow if operating_net_cash_flow else None

def cash_debt_coverage_ratio(operating_net_cash_flow, avg_total_liabilities):
    return operating_net_cash_flow / avg_total_liabilities if avg_total_liabilities else None

def cash_interest_coverage(operating_net_cash_flow, interest_expense):
    return (operating_net_cash_flow + interest_expense) / interest_expense if interest_expense else None

def cash_return_on_assets(operating_net_cash_flow, avg_total_assets):
    return operating_net_cash_flow / avg_total_assets if avg_total_assets else None

def cash_return_on_fixed_assets(operating_net_cash_flow, avg_fixed_assets):
    return operating_net_cash_flow / avg_fixed_assets if avg_fixed_assets else None

def CFO_to_total_equity(operating_net_cash_flow, avg_total_equity):
    return operating_net_cash_flow / avg_total_equity if avg_total_equity else None

def cash_flow_from_sales_to_sales(operating_net_cash_flow, net_operating_sales):
    return operating_net_cash_flow / net_operating_sales if net_operating_sales else None

def cash_flow_margin(operating_net_cash_flow, total_revenue):
    return operating_net_cash_flow / total_revenue if total_revenue else None

def earning_quality_ratio(operating_net_cash_flow, net_income):
    return operating_net_cash_flow / net_income if net_income else None

# Dictionary to map ratios to category codes (with average calculations for avg fields)
cash_flow_ratio_functions_6 = {
    'EBITDA': [['IS_050', 'IS_023'], 'CF_002'],  # EBIT (IS_050), depreciation_and_amortization (CF_002)
    'free_cash_flow': ['CF_020', ['CF_021', 'CF_023'], 'CF_036'],  # operating_net_cash_flow (CF_020), capital_expenditures (CF_021 + CF_023), dividends_paid (CF_036)
    'free_cash_flow_to_operating_cash_flow_ratio': ['free_cash_flow', 'CF_020'],  # free_cash_flow, operating_net_cash_flow (CF_020)
    'cash_debt_coverage_ratio': ['CF_020', 'BS_300'],  # operating_net_cash_flow (CF_020), avg_total_liabilities (BS_300)
    'cash_interest_coverage': ['CF_020', 'IS_023'],  # operating_net_cash_flow (CF_020), interest_expense (IS_023)
    'cash_return_on_assets': ['CF_020', 'BS_270'],  # operating_net_cash_flow (CF_020), avg_total_assets (BS_270)
    'cash_return_on_fixed_assets': ['CF_020', 'BS_220'],  # operating_net_cash_flow (CF_020), avg_fixed_assets (BS_220)
    'CFO_to_total_equity': ['CF_020', 'BS_400'],  # operating_net_cash_flow (CF_020), avg_total_equity (BS_400)
    'cash_flow_from_sales_to_sales': ['CF_020', 'IS_010'],  # operating_net_cash_flow (CF_020), net_sales (IS_010+IS_021)
    'cash_flow_margin': ['CF_020', ['IS_010', 'IS_021']],  # operating_net_cash_flow (CF_020), total_revenue (IS_010 + IS_021)
    'earning_quality_ratio': ['CF_020', 'IS_060'],  # operating_net_cash_flow (CF_020), net_income (IS_060)
}

# Create a DataFrame to store the results of the new cash flow ratios
cash_flow_results_6 = []

# Iterate through the pivot table to calculate the cash flow ratios
for index, row in pivot_df_6.iterrows():
    stock_code, year, quarter = index
    
    for ratio, inputs in cash_flow_ratio_functions_6.items():
        input_values = []
        for input_name in inputs:
            if isinstance(input_name, list):  # Sum for cases like capital_expenditures or total_revenue
                value_sum = sum([row[i] for i in input_name if i in row.index])
                input_values.append(value_sum)
            else:
                if input_name in ['BS_300', 'BS_270', 'BS_220', 'BS_400']:  
                    prev_q0_value = get_previous_year_q0_value(stock_code, year, input_name)
                    current_value = row[input_name] if input_name in row.index else None
                    if current_value is not None and prev_q0_value is not None:
                        avg_value = (current_value + prev_q0_value) / 2
                        input_values.append(avg_value)
                    else:
                        input_values.append(None)
                else:
                    input_values.append(row[input_name] if input_name in row.index else None)
        
        # Check if all required data is available
        if None not in input_values:
            # Call the corresponding function to calculate the ratio
            ratio_value = globals()[ratio](*input_values)
            cash_flow_results_6.append({
                'stock_code': stock_code,
                'year': year,
                'quarter': quarter,
                'ratio_code': ratio,
                'data': ratio_value
            })

# Convert the results to a DataFrame
ratios_df_6 = pd.DataFrame(cash_flow_results_6)

# Display or export the DataFrame
ratios_df_6


,stock_code,year,quarter,ratio_code,data
0,ACV,2015,0,EBITDA,5.798844e+06
1,ACV,2015,0,free_cash_flow,6.817295e+06
2,ACV,2015,0,cash_interest_coverage,-4.588455e+01
3,ACV,2015,0,cash_flow_from_sales_to_sales,3.198536e-01
4,ACV,2015,0,cash_flow_margin,2.981326e-01
...,...,...,...,...,...
35331,YEG,2024,3,cash_return_on_fixed_assets,-5.656272e-01
35332,YEG,2024,3,CFO_to_total_equity,-5.288747e-02
35333,YEG,2024,3,cash_flow_from_sales_to_sales,-2.159746e-01
35334,YEG,2024,3,cash_flow_margin,-2.016505e-01


# YoY ratio

In [14]:
import pandas as pd

# Pivot table to restructure financial data based on category codes
pivot_df_7 = data_df.pivot_table(index=['stock_code', 'year', 'quarter'], 
                               columns='category_code', 
                               values='data', 
                               aggfunc='sum')

# Function to calculate YoY growth
def yoy_growth(current_year_value, previous_year_value):
    return (current_year_value - previous_year_value) / previous_year_value if previous_year_value else None

# Mapping of ratios to their respective category codes
ratio_mapping = {
    'Net_Revenue_Growth_YoY': 'IS_010',
    'Gross_Profit_Growth_YoY': 'IS_020',
    'EBITDA_Growth_YoY': 'EBITDA',
    'EBIT_Growth_YoY': 'EBIT',
    'Pre_Tax_Profit_Growth_YoY': 'IS_050',
    'Accounts_Receivable_Growth_YoY': ['BS_131', 'BS_211'],  # Summing up these codes
    'Inventory_Growth_YoY': 'BS_140',
    'Short_Term_Debt_Growth_YoY': 'BS_320',
    'Long_Term_Debt_Growth_YoY': 'BS_338',
    'SG&A_Expense_Growth_YoY': ['IS_025', 'IS_026'],  # Summing up these codes
    'Total_Asset_Growth_YoY': 'BS_270',
    'Equity_Growth_YoY': 'BS_400',
    'CFO_Growth_YoY': 'CF_020'
}

# Helper function to fetch pre-calculated ratios from ratios_df_6
def get_pre_calculated_ratio(stock_code, year, ratio_code, ratios_df, quarter):
    try:
        return ratios_df.loc[(ratios_df['stock_code'] == stock_code) & 
                              (ratios_df['year'] == year) & 
                              (ratios_df['quarter'] == quarter) & 
                              (ratios_df['ratio_code'] == ratio_code), 
                              'data'].values[0]
    except (IndexError, KeyError):
        return None

# Create a DataFrame to store the results
results_7 = []

# Iterate through the pivoted DataFrame
for (stock_code, year, quarter), row in pivot_df_7.iterrows():
        
    # Process YoY growth calculation for each quarter dynamically
    for ratio_name, category_code in ratio_mapping.items():
        # Fetch current and previous year values dynamically
        if isinstance(category_code, list):
            # Handle sums of multiple category codes
            current_year_value = sum(row.get(code, 0) for code in category_code)
            previous_year_value = sum(
                pivot_df_7.loc[(stock_code, year - 1, quarter), code]
                if (stock_code, year - 1, quarter) in pivot_df_7.index and code in pivot_df_7.columns
                else 0 for code in category_code
            )
        elif category_code in ['EBIT']:
            current_year_value = get_pre_calculated_ratio(stock_code, year, category_code, ratios_df_1, quarter)
            previous_year_value = get_pre_calculated_ratio(stock_code, year - 1, category_code, ratios_df_1, quarter)
        elif category_code in ['EBITDA']:
            current_year_value = get_pre_calculated_ratio(stock_code, year, category_code, ratios_df_6, quarter)
            previous_year_value = get_pre_calculated_ratio(stock_code, year - 1, category_code, ratios_df_6, quarter)
        else:
            # Standard case: Fetch current and previous year's values for the same quarter
            current_year_value = row.get(category_code, 0)
            previous_year_value = pivot_df_7.loc[
                (stock_code, year - 1, quarter), category_code
            ] if (stock_code, year - 1, quarter) in pivot_df_7.index else 0
        
        # Calculate YoY growth
        yoy_value = None
        if previous_year_value == 0 or previous_year_value is None:
            yoy_value = None  # Avoid division by zero
        else:
            yoy_value = (current_year_value - previous_year_value) / previous_year_value

        # Store results
        results_7.append({
            'stock_code': stock_code,
            'year': year,
            'quarter': quarter,
            'ratio_code': ratio_name,
            'data': yoy_value
        })
# Convert results to a DataFrame
ratios_df_7 = pd.DataFrame(results_7)



In [15]:
ratios_df_7

,stock_code,year,quarter,ratio_code,data
0,ACV,2015,0,Net_Revenue_Growth_YoY,NaN
1,ACV,2015,0,Gross_Profit_Growth_YoY,NaN
2,ACV,2015,0,EBITDA_Growth_YoY,NaN
3,ACV,2015,0,EBIT_Growth_YoY,NaN
4,ACV,2015,0,Pre_Tax_Profit_Growth_YoY,NaN
...,...,...,...,...,...
47679,YEG,2024,3,Long_Term_Debt_Growth_YoY,-0.357909
47680,YEG,2024,3,SG&A_Expense_Growth_YoY,1.386906
47681,YEG,2024,3,Total_Asset_Growth_YoY,0.308467
47682,YEG,2024,3,Equity_Growth_YoY,0.047895


In [20]:
ratios_df_7[(ratios_df_7['stock_code'] == 'YEG') & (ratios_df_7['ratio_code'] == 'SG&A_Expense_Growth_YoY') & (ratios_df_7['year'] == 2017) & (ratios_df_7['quarter'] == 0)]

,stock_code,year,quarter,ratio_code,data
7239,YEG,2017,0,SG&A_Expense_Growth_YoY,0.180143


In [21]:
ratios_df_6[(ratios_df_6['stock_code'] == 'ACV') & (ratios_df_6['ratio_code'] == 'EBITDA') & (ratios_df_6['year'] == 2016) & (ratios_df_6['quarter'] == 0)]

,stock_code,year,quarter,ratio_code,data
0,ACV,2016,0,EBITDA,1.078568e+07


In [22]:
ratios_df_6[(ratios_df_6['stock_code'] == 'ACV') & (ratios_df_6['ratio_code'] == 'EBITDA') & (ratios_df_6['year'] == 2017) & (ratios_df_6['quarter'] == 0)]

,stock_code,year,quarter,ratio_code,data
30,ACV,2017,0,EBITDA,9.412331e+06


In [23]:
ratios_df_6[(ratios_df_6['stock_code'] == 'ACV') & (ratios_df_6['ratio_code'] == 'EBITDA') & (ratios_df_6['year'] == 2016) & (ratios_df_6['quarter'] == 0)]

,stock_code,year,quarter,ratio_code,data
0,ACV,2016,0,EBITDA,1.078568e+07


In [24]:
data_df[(data_df['category_code'] == 'IS_025') & (data_df['year'] == 2017) &(data_df['stock_code'] == 'YEG') & (data_df['quarter'] == 0)]

,category_code,data,stock_code,quarter,year,date_added
38661,IS_025,-52734.972406,YEG,0,2017,2017-12-30


In [25]:
data_df[(data_df['category_code'] == 'IS_026') & (data_df['year'] == 2017) &(data_df['stock_code'] == 'YEG') & (data_df['quarter'] == 0)]

,category_code,data,stock_code,quarter,year,date_added
38669,IS_026,-85596.057685,YEG,0,2017,2017-12-30


In [26]:
# Concatenate all the DataFrames
ratios_df = pd.concat([ratios_df_1, ratios_df_2, ratios_df_3, ratios_df_4, ratios_df_5, ratios_df_6, ratios_df_7], ignore_index=True)

# Display the concatenated DataFrame
ratios_df

,stock_code,year,quarter,ratio_code,data
0,ACV,2016,0,EBIT,6.569465e+06
1,ACV,2016,0,equity_ratio,5.183555e-01
2,ACV,2016,0,long_term_asset_self_financing_ratio,1.557680e+00
3,ACV,2016,0,fixed_asset_self_financing_ratio,1.688407e+00
4,ACV,2016,0,general_solvency_ratio,2.076220e+00
...,...,...,...,...,...
182517,YEG,2023,0,Long_Term_Debt_Growth_YoY,3.801443e+01
182518,YEG,2023,0,SG&A_Expense_Growth_YoY,-2.883296e-01
182519,YEG,2023,0,Total_Asset_Growth_YoY,4.991388e-01
182520,YEG,2023,0,Equity_Growth_YoY,5.255066e-01


In [27]:
ratios_df[(ratios_df['stock_code'] == 'VIC')&(ratios_df['year'] == 2020)&(ratios_df['quarter'] == 0)&(ratios_df['ratio_code'] == 'CFO_Growth_YoY')]

,stock_code,year,quarter,ratio_code,data
181831,VIC,2020,0,CFO_Growth_YoY,-0.000762


In [16]:
df = pd.read_parquet(r'D:\python\financial statement prj\chatbot_financial_statement\data\financial_ratio_v3.parquet')
df

,stock_code,year,quarter,ratio_code,data,date_added
0,ACV,2015,0,EBIT,2.187559e+06,2015-12-30
1,ACV,2015,0,ER,4.596180e-01,2015-12-30
2,ACV,2015,0,LTASFR,1.593160e+00,2015-12-30
3,ACV,2015,0,FASFR,1.690095e+00,2015-12-30
4,ACV,2015,0,GSR,1.850543e+00,2015-12-30
...,...,...,...,...,...,...
357388,VND,2024,1,DPO,NaN,2024-03-30
357389,VND,2024,2,DSO,NaN,2024-06-30
357390,VND,2024,2,DPO,NaN,2024-06-30
357391,VND,2024,3,DSO,NaN,2024-09-30


In [ ]:
df[df['ratio_code'] == 'ROAA']

,stock_code,year,quarter,ratio_code,data,date_added
198072,ACV,2015,0,ROAA,0.039087,2015-12-30
198075,ACV,2016,0,ROAA,0.112695,2016-12-30
198078,ACV,2016,1,ROAA,0.054691,2016-03-30
198081,ACV,2016,2,ROAA,-0.000356,2016-06-30
198084,ACV,2016,3,ROAA,0.017610,2016-09-30
...,...,...,...,...,...,...
353298,VND,2023,3,ROAA,0.015891,2023-09-30
353301,VND,2023,4,ROAA,0.020353,2023-12-30
353304,VND,2024,1,ROAA,0.014853,2024-03-30
353307,VND,2024,2,ROAA,0.007938,2024-06-30


: 

array(['EBIT', 'ER', 'LTASFR', 'FASFR', 'GSR', 'ROI', 'ROIC', 'BEP',
       'DTAR', 'DTER', 'STDTAR', 'ICR', 'LTDTER', 'STDTER', 'RTPR',
       'RTTAR', 'DTTCR', 'RTSR', 'AFDAR', 'ATDR', 'CurrentR', 'QR',
       'CashR', 'LTDCR', 'LTDTECR', 'TIE', 'DTTNWR', 'FL', 'AFDATTAR',
       'PFR', 'FITNRR', 'ROA', 'ROFA', 'ROLTOA', 'BEPR', 'ROE', 'POCOGS',
       'PSR', 'POOE', 'ROS', 'OPM', 'NPM', 'GPM', 'TAT', 'EBITDA', 'FCF',
       'CDCR', 'CIC', 'CROA', 'CROFA', 'CFOTTE', 'CFFSTS', 'CFM', 'EQR',
       'ROAA', 'ROAE', 'ROAS', 'EPS', 'PE', 'BVPS', 'PB', 'NRGYoY',
       'GPGYoY', 'EBITDAGYoY', 'EBITGYoY', 'PTPGYoY', 'ARGYoY', 'IGYoY',
       'STDGYoY', 'LTDGYoY', 'SG&AEGYoY', 'TAGYoY', 'EGYoY', 'CFOGYoY',
       'ITR', 'DSO', 'DPO', 'DIO', 'CCC', 'ALCR', 'NIM', 'CDGYoY',
       'OEGYoY', 'IBPGYoY', 'IIGYoY', 'NIIGYoY', 'TOIGYoY', 'CLGYoY',
       'CASA', 'BDR'], dtype=object)